In [5]:
%cd ..
!pwd

/home/ubuntu/Study/Otus/ml-finance/homeworks/OTUS-HW1
/home/ubuntu/Study/Otus/ml-finance/homeworks/OTUS-HW1


In [6]:
from pathlib import Path
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.tsa.stattools import adfuller
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from sklearn.ensemble import IsolationForest
from sklearn.neighbors import LocalOutlierFactor
from loguru import logger  # type: ignore
from tqdm import tqdm  # type: ignore

from otus_hw1.config import RAW_DATA_DIR

2024-12-02 18:01:22.702 | INFO     | otus_hw1.config:<module>:11 - PROJ_ROOT path is: /home/ubuntu/Study/Otus/ml-finance/homeworks/OTUS-HW1


In [8]:
df = pd.read_csv(RAW_DATA_DIR/"sp500_data/AAPL.csv")
df

,Date,Adj Close,Close,High,Low,Open,Volume
0,2023-11-30,189.023956,189.949997,190.320007,188.190002,189.839996,48794400
1,2023-12-01,190.307678,191.240005,191.559998,189.229996,190.330002,45679300
2,2023-12-04,188.506470,189.429993,190.050003,187.449997,189.979996,43389500
3,2023-12-05,192.477051,193.419998,194.399994,190.179993,190.210007,66628400
4,2023-12-06,191.382416,192.320007,194.759995,192.110001,194.449997,41089700
...,...,...,...,...,...,...,...
247,2024-11-22,229.869995,229.869995,230.720001,228.059998,228.059998,38168300
248,2024-11-25,232.869995,232.869995,233.250000,229.740005,231.460007,90152800
249,2024-11-26,235.059998,235.059998,235.570007,233.330002,233.330002,45986200
250,2024-11-27,234.929993,234.929993,235.690002,233.809998,234.470001,33498400


In [10]:
# Функция для проверки данных на пропуски и ошибки
def check_missing_and_errors(df: pd.DataFrame) -> pd.DataFrame:
    """
    Проверяет пропуски и ошибки в данных.
    """
    logger.info("Проверка пропусков...")
    missing_info = df.isnull().sum()
    if missing_info.any():
        logger.warning(f"Найдены пропуски:\n{missing_info}")
    else:
        logger.success("Пропуски не найдены.")

    logger.info("Проверка логических ошибок...")
    error_rows = df[
        (df['Low'] > df['Open']) |
        (df['Open'] > df['High']) |
        (df['Low'] > df['Close']) |
        (df['Close'] > df['High']) |
        (df['Volume'] < 0)
    ]
    if not error_rows.empty:
        logger.warning(f"Найдены логические ошибки:\n{error_rows}")
    else:
        logger.success("Логических ошибок не найдено.")

    logger.info("Проверка временных интервалов...")
    df['TimeDiff'] = df['Date'].diff()
    irregular_intervals = df[df['TimeDiff'] != df['TimeDiff'].median()]
    if not irregular_intervals.empty:
        logger.warning(f"Обнаружены нерегулярные интервалы:\n{irregular_intervals}")
    else:
        logger.success("Временные интервалы равномерны.")
    return df

check_missing_and_errors(df)

2024-12-02 18:02:44.068 | INFO     | __main__:check_missing_and_errors:6 - Проверка пропусков...
2024-12-02 18:02:44.070 | SUCCESS  | __main__:check_missing_and_errors:11 - Пропуски не найдены.
2024-12-02 18:02:44.070 | INFO     | __main__:check_missing_and_errors:13 - Проверка логических ошибок...
2024-12-02 18:02:44.072 | SUCCESS  | __main__:check_missing_and_errors:24 - Логических ошибок не найдено.
2024-12-02 18:02:44.072 | INFO     | __main__:check_missing_and_errors:26 - Проверка временных интервалов...


TypeError: unsupported operand type(s) for -: 'str' and 'str'

In [ ]:


# Анализ выбросов
def analyze_outliers(df: pd.DataFrame) -> pd.DataFrame:
    """
    Анализ выбросов: IQR, Z-оценка, Local Outlier Factor, Isolation Forest.
    """
    logger.info("Анализ выбросов...")
    # IQR
    q1 = df[['Open', 'High', 'Low', 'Close']].quantile(0.25)
    q3 = df[['Open', 'High', 'Low', 'Close']].quantile(0.75)
    iqr = q3 - q1
    outliers_iqr = (df[['Open', 'High', 'Low', 'Close']] < (q1 - 1.5 * iqr)) | \
                   (df[['Open', 'High', 'Low', 'Close']] > (q3 + 1.5 * iqr))
    df['Outlier_IQR'] = outliers_iqr.any(axis=1)
    logger.info(f"IQR выбросы обнаружены: {df['Outlier_IQR'].sum()}")

    # Local Outlier Factor
    lof = LocalOutlierFactor(n_neighbors=20)
    df['Outlier_LOF'] = lof.fit_predict(df[['Open', 'High', 'Low', 'Close', 'Volume']]) == -1
    logger.info(f"LOF выбросы обнаружены: {df['Outlier_LOF'].sum()}")

    # Isolation Forest
    iso_forest = IsolationForest(contamination=0.01, random_state=42)
    df['Outlier_IsolationForest'] = iso_forest.fit_predict(df[['Open', 'High', 'Low', 'Close', 'Volume']]) == -1
    logger.info(f"Isolation Forest выбросы обнаружены: {df['Outlier_IsolationForest'].sum()}")

    return df

# Сохранение аномальных отрезков
def save_anomalies_visual(df: pd.DataFrame, column: str, output_dir: Path):
    """
    Сохраняет графики аномальных отрезков.
    """
    anomalies = df[df['Outlier_IQR'] | df['Outlier_LOF'] | df['Outlier_IsolationForest']]
    for i, idx in enumerate(anomalies.index[:5]):  # Сохраним только 5 первых аномалий
        start = max(0, idx - 25)
        end = min(len(df), idx + 25)
        segment = df.iloc[start:end]
        plt.figure(figsize=(10, 6))
        plt.plot(segment['Date'], segment[column], label=f'{column}')
        plt.title(f"Аномалия {i + 1} в {column}")
        plt.legend()
        plt.savefig(output_dir / f'anomaly_{column}_{i + 1}.png')
        plt.close()

# Анализ временных рядов
def time_series_analysis(df: pd.DataFrame, column: str):
    """
    Анализ временных рядов: ACF, PACF, тест Дики-Фуллера.
    """
    logger.info("Проверка стационарности временного ряда...")
    adf_result = adfuller(df[column].dropna())
    logger.info(f"ADF Statistic: {adf_result[0]}")
    logger.info(f"p-value: {adf_result[1]}")
    if adf_result[1] < 0.05:
        logger.success(f"Ряд {column} стационарен.")
    else:
        logger.warning(f"Ряд {column} нестационарен.")

    logger.info("Построение ACF и PACF...")
    plt.figure(figsize=(12, 6))
    plot_acf(df[column].dropna(), lags=50)
    plt.title(f"ACF для {column}")
    plt.show()

    plt.figure(figsize=(12, 6))
    plot_pacf(df[column].dropna(), lags=50)
    plt.title(f"PACF для {column}")
    plt.show()

# Пример использования
if __name__ == "__main__":
    # Укажите пути к данным и выходной папке
    input_file = Path("ohlc_data.csv")  # Ваш файл с данными
    output_dir = Path("output_anomalies")
    output_dir.mkdir(parents=True, exist_ok=True)

    # Загрузка данных
    df = pd.read_csv(input_file)
    df['Date'] = pd.to_datetime(df['Date'])

    # Проверка данных
    df = check_missing_and_errors(df)

    # Анализ выбросов
    df = analyze_outliers(df)

    # Сохранение аномальных участков
    save_anomalies_visual(df, 'Close', output_dir)

    # Анализ временного ряда
    time_series_analysis(df, 'Close')

    # Сохранение результатов
    df.to_csv(output_dir / "analyzed_data.csv", index=False)
    logger.success("Анализ завершен. Результаты сохранены.")
